In [4]:
import pandas as pd 
import numpy as np
import quandl
import datetime



In [16]:
recap = pd.DataFrame(index =["Valeur absolue","Distance Euclidienne","Score3"],columns=['GARCH', 'ARIMA', 'ML1', 'ML2', 'ML3']).fillna(0)
recap

,GARCH,ARIMA,ML1,ML2,ML3
Valeur absolue,0,0,0,0,0
Distance Euclidienne,0,0,0,0,0
Score3,0,0,0,0,0


In [11]:
df = pd.read_excel(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\std25_test.xlsx", sheetname='Sheet1')

In [12]:
Ecartabs=abs(df['MSFT']-df["Prédiction"]).dropna()
Ecartdist=((df['MSFT']-df["Prédiction"])**2).dropna()

In [21]:
# Vabs=Ecartabs.mean()
# Vdist=(Ecartdist.mean())**(0.5)
# (Vabs, Vdist)

# Données Financières

In [117]:
S = pd.read_excel(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Entreprises_S&P.xlsx", sheetname='Feuil1')
logo=list(S['Logo'])[:3] #3 mi ici pour une question de temps
start = datetime.datetime(2005,1,1)
end = datetime.date.today()

In [125]:
data_fi=pd.DataFrame({logo[0] : quandl.get("WIKI/" +logo[0] , start_date=start, end_date=end)['Adj. Close']})

for l in logo[1:]:
    data=pd.DataFrame({l : quandl.get("WIKI/" +l , start_date=start, end_date=end)['Adj. Close']})
    data_fi=data_fi.join(data)
    

#data_fi.to_excel('Data Financière.xlsx')
#data_fi.to_csv('Data Financière.csv')


# Données Simmulées

# Forecast

In [22]:
#Que les méthode de forecast enregistre le tout dans un csv
# ex :
# garch = forecast_garch(L[0])
# for l in logo[1:]:
#     temp= forecast_garch(l)
#     garch=garch.join(temp)

# garch.to_csv('GARCH.CSV')

# De même avec les autres méthode


In [ ]:
# garch = pandas.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\GARCH.csv)
# arima = pandas.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\ARIMA.csv)
# ml1 = pandas.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\ML1.csv)                       
# ml2 = pandas.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\ML2.csv)
# ml3 = pandas.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\ML3.csv)

# Détermination du score pour Data Financière

###  Valeur Absolue

#### GARCH

In [ ]:
list_abs=[]
for l in logo:
    ecart = abs(data_fi[l]-garch[l]).dropna().mean()
    list_abs.append(ecart)
    
Score_abs = mean(list_abs)

recap['GARCH']['Valeur absolue']= score_abs

# Y à t'il un moyen de créer une méthode auto(score,Méthode de forecast,type de data) pour pas avoir à le refaire pour tous les différents score et méthodes et type de data (simmulé ou réelle)

